In [1]:
import pandas as pd 

df = pd.read_csv("/kaggle/input/darwin/data.csv")
df = df.drop(columns=["ID"])
df['class'] = df['class'].replace({'P':1,'H':0})
df.head()

/tmp/ipykernel_18/1265945748.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['class'] = df['class'].replace({'P':1,'H':0})


,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,mean_jerk_in_air1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,0.051836,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,1
1,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,0.039827,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,1
2,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,0.064220,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,1
3,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,0.090408,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,1
4,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,0.037528,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,1


# Cross-Validation (have no clue wtf was it)

In [2]:
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

y = df['class']
X = df.drop(columns=['class'])

k = 7
model = LinearRegression()
kf = KFold(n_splits = k, shuffle = True, random_state = 42)
results = cross_val_score(model, X,y, cv=kf, scoring="r2")

print(results.mean())
print(results.std())



-21.958842908053303
34.134028852998696


# Holdout Validation (linear regression)

In [3]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

r2 = r2_score(y_test,y_pred)
print(r2)


-448.41961698249963


(polynomial regression) 

In [4]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

degree = 2
f = PolynomialFeatures(degree = degree)
X_poly_train = f.fit_transform(X_train)
X_poly_test = f.transform(X_test)

model1 = make_pipeline(PolynomialFeatures(degree), LinearRegression())

model1.fit(X_train, y_train)

y_pred = model.predict(X_test)

r2 = r2_score(y_test,y_pred)
print(r2)


-448.41961698249963


# PCA (this is underfitting now ah.)

In [5]:
# from sklearn.decomposition import PCA 
# X = pd.DataFrame(X)
# y = pd.DataFrame(y)

# k = 5

# pca = PCA(n_components=0.95)  # Retain 95% of variance
# model = LinearRegression()
# pipeline = make_pipeline(pca, model)


# kf = KFold(n_splits=k, shuffle=True, random_state=42)
# cv_results = []

# for train_index, test_index in kf.split(X):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     pipeline.fit(X_train, y_train)
#     y_pred = pipeline.predict(X_test)
#     r2 = r2_score(y_test, y_pred)
#     cv_results.append(r2)

# cv_results = np.array(cv_results)
# print("Mean R^2 Score:", cv_results.mean())
# print("Standard Deviation of R^2 Score:", cv_results.std())

# Let's try random forest and gradient boosting

In [6]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import numpy as np

# Convert X and y to DataFrames if they are numpy arrays
X = pd.DataFrame(X)
y = pd.DataFrame(y)

# Define the number of folds for cross-validation
k = 5

# Create PCA transformer
pca = PCA(n_components=0.95)  # Retain 95% of variance

# Create Random Forest and Gradient Boosting models
rf_model = RandomForestRegressor(n_estimators=50, max_depth = None, min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Perform k-fold cross-validation
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize lists to store R-squared scores
rf_cv_results = []
gb_cv_results = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit PCA on training data and transform training/testing data
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    # Fit Random Forest model on PCA-transformed data
    rf_model.fit(X_train_pca, y_train.values.ravel())
    # Make predictions with Random Forest model
    rf_y_pred = rf_model.predict(X_test_pca)
    # Calculate R-squared score for Random Forest model
    rf_r2 = r2_score(y_test, rf_y_pred)
    rf_cv_results.append(rf_r2)
    
    # Fit Gradient Boosting model on PCA-transformed data
    gb_model.fit(X_train_pca, y_train.values.ravel())
    # Make predictions with Gradient Boosting model
    gb_y_pred = gb_model.predict(X_test_pca)
    # Calculate R-squared score for Gradient Boosting model
    gb_r2 = r2_score(y_test, gb_y_pred)
    gb_cv_results.append(gb_r2)

# Convert lists to numpy arrays
rf_cv_results = np.array(rf_cv_results)
gb_cv_results = np.array(gb_cv_results)

# Print mean and standard deviation of R-squared scores for Random Forest
print("Random Forest:")
print("Mean R^2 Score:", rf_cv_results.mean())
print("Standard Deviation of R^2 Score:", rf_cv_results.std())

# Print mean and standard deviation of R-squared scores for Gradient Boosting
print("\nGradient Boosting:")
print("Mean R^2 Score:", gb_cv_results.mean())
print("Standard Deviation of R^2 Score:", gb_cv_results.std())


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

Random Forest:
Mean R^2 Score: 0.3121288015135879
Standard Deviation of R^2 Score: 0.07789541989786727

Gradient Boosting:
Mean R^2 Score: 0.13656697727775471
Standard Deviation of R^2 Score: 0.14605985888504253
